In [1]:
try:
    from pathlib import Path
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from tqdm import tqdm
    import numpy as np
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from tqdm import tqdm
    from matplotlib.ticker import MaxNLocator
    import cv2
    import json
    import pywt
    from matplotlib.colors import LinearSegmentedColormap
    import shap
    from lib.utils import inverti_maschera_binaria
    from PIL import Image, ImageOps
except Exception as e:
    print(f"Some module are missing: {e}\n")

In [2]:
def calculate_entropy(data: np.ndarray, bins: int = 256) -> float:
    data_flat = data.flatten()
    histogram, _ = np.histogram(data_flat, bins=bins, range=(0, bins), density=True)
    entropy = -np.sum(histogram * np.log2(histogram + 1e-10))
    return entropy


def clalc_shift_spect(img):
    f = np.fft.fft2(img)

    coeffs = pywt.wavedec2(img, "haar", level=2)
    return f, coeffs


def extract_frequency_features(
    image: Path | np.ndarray, wavelet: str = "db4", bins: int = 256, invert_mask=False
) -> dict:
    image = image.resolve()
    if isinstance(image, Path):
        img = cv2.imread(str(image), cv2.IMREAD_GRAYSCALE)
    else:
        img = image.copy()
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
    if invert_mask:
        img = cv2.bitwise_not(img)

    # f = np.fft.fft2(img)
    f, coeffs = clalc_shift_spect(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-10)

    mean_freq = np.mean(magnitude_spectrum)
    std_freq = np.std(magnitude_spectrum)
    max_freq = np.max(magnitude_spectrum)
    min_freq = np.min(magnitude_spectrum)
    median_freq = np.median(magnitude_spectrum)
    energy = np.sum(np.abs(fshift) ** 2)

    rows, cols = magnitude_spectrum.shape
    crow, ccol = rows // 2, cols // 2

    histogram, _ = np.histogram(
        magnitude_spectrum, bins=bins, range=(0, bins), density=True
    )
    entropy = -np.sum(histogram * np.log2(histogram + 1e-10))

    low_freq_energy = np.sum(magnitude_spectrum[:crow, :ccol])
    high_freq_energy = np.sum(magnitude_spectrum[crow:, ccol:])
    frequency_contrast = high_freq_energy - low_freq_energy

    # coeffs = pywt.wavedec2(img, wavelet, level=2)
    cA2, (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs

    wavelet_features = {
        "wavelet_energy_A2": np.sum(np.square(cA2)),
        "wavelet_energy_H2": np.sum(np.square(cH2)),
        "wavelet_energy_V2": np.sum(np.square(cV2)),
        "wavelet_energy_D2": np.sum(np.square(cD2)),
        "wavelet_std_A2": np.std(cA2),
        "wavelet_std_H2": np.std(cH2),
        "wavelet_std_V2": np.std(cV2),
        "wavelet_std_D2": np.std(cD2),
        "wavelet_energy_H1": np.sum(np.square(cH1)),
        "wavelet_energy_V1": np.sum(np.square(cV1)),
        "wavelet_energy_D1": np.sum(np.square(cD1)),
        "wavelet_std_H1": np.std(cH1),
        "wavelet_std_V1": np.std(cV1),
        "wavelet_std_D1": np.std(cD1),
        "wavelet_entropy_A2": calculate_entropy(cA2, bins),
        "wavelet_entropy_H2": calculate_entropy(cH2, bins),
        "wavelet_entropy_V2": calculate_entropy(cV2, bins),
        "wavelet_entropy_D2": calculate_entropy(cD2, bins),
        "wavelet_entropy_H1": calculate_entropy(cH1, bins),
        "wavelet_entropy_V1": calculate_entropy(cV1, bins),
        "wavelet_entropy_D1": calculate_entropy(cD1, bins),
    }

    frequency_features = {
        "mean_frequency": mean_freq,
        "std_frequency": std_freq,
        "max_frequency": max_freq,
        "min_frequency": min_freq,
        "median_frequency": median_freq,
        "energy": energy,
        "entropy": entropy,
        "low_frequency_energy": low_freq_energy,
        "high_frequency_energy": high_freq_energy,
        "frequency_contrast": frequency_contrast,
    }

    frequency_features.update(wavelet_features)

    return frequency_features

In [4]:
#images_path=Path("/home/tommaso/git_workspace/VAE_DefectedGraphene/results/generated_test")
images_path=Path("/home/tommaso/git_workspace/VAE_DefectedGraphene/results/26_10/generated_dataset")
frequency_dict = {}

images = [f for f in images_path.iterdir() if f.suffix.lower() ==".png"]
for image in tqdm(images):
    dict_img = extract_frequency_features(image, invert_mask=True)
    if int(np.loadtxt(image.with_suffix(".txt"))) >= 512:
        print("Warning!")
        continue
    else:
        dict_img["n_atoms"] = int(np.loadtxt(image.with_suffix(".txt")))
        dict_img["file_name"] = image.stem
        frequency_dict[f"{image.stem}"] = dict_img

frequency_df = pd.DataFrame.from_dict(frequency_dict, orient="index")

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:32<00:00, 151.95it/s]


In [5]:
# # Supponiamo che tu abbia un DataFrame chiamato df con le tue features
# correlation_matrix = frequency_df.corr()

# # Visualizza la matrice di correlazione come una heatmap
# plt.figure(figsize=(15, 15))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
# plt.show()

In [6]:
def padding_image(image, size=240):
    h = image.shape[0]
    w = image.shape[1]

    top = round((size - h) / 2)
    bottom = size - (h + top)

    left = round((size - w) / 2)
    right = size - (w + left)

    padded_img = cv2.copyMakeBorder(
        image, top, bottom, left, right, cv2.BORDER_CONSTANT, None, 255
    )

    return padded_img

In [7]:
def conta_atomi_da_xyz(spath: Path):
        """Read xyz files and return lists of x,y,z coordinates and atoms"""

        atoms = []

        with open(str(spath), "r") as f:
            for line in f:
                l = line.split()
                if len(l) == 4 or len(l) == 5:
                    atoms.append(str(l[0]))

        return len(atoms)

In [46]:
# images_path=Path("/home/tommaso/git_workspace/VAE_DefectedGraphene/data/images_240")
# images = [f for f in images_path.iterdir() if f.suffix.lower() ==".png"]
# for image in images:
#     img=inverti_maschera_binaria(str(image))
#     img.save(Path(f"/home/tommaso/git_workspace/VAE_DefectedGraphene/analysis/reverted_dataset_image/{image.name}"))
    
# images_path=Path("/home/tommaso/git_workspace/VAE_DefectedGraphene/analysis/reverted_dataset_image")
# images = [f for f in images_path.iterdir() if f.suffix.lower() ==".png"] 
# for image in images:
#     img = cv2.imread(str(image),0)
#     img = padding_image(img,size=240)
#     cv2.imwrite(str(Path(f"/home/tommaso/git_workspace/VAE_DefectedGraphene/analysis/padded/{image.name}")),img)

In [8]:
images_path=Path("/home/tommaso/git_workspace/VAE_DefectedGraphene/analysis/padded")
frequency_dict = {}

images = [f for f in images_path.iterdir() if f.suffix.lower() ==".png"]
for image in tqdm(images):
    dict_img = extract_frequency_features(image, invert_mask=True)
    dict_img["n_atoms"] = conta_atomi_da_xyz(Path(f"/home/tommaso/git_workspace/VAE_DefectedGraphene/data/xyz_files/{image.stem}.xyz"))
    dict_img["file_name"] = image.stem
    frequency_dict[f"{image.stem}"] = dict_img

original_frequency_df = pd.DataFrame.from_dict(frequency_dict, orient="index")

  0%|          | 0/8750 [00:00<?, ?it/s]

100%|██████████| 8750/8750 [01:02<00:00, 139.38it/s]


In [9]:
frequency_df.to_csv(f"/home/tommaso/git_workspace/VAE_DefectedGraphene/analysis/generated_df.csv", index=False)
original_frequency_df.to_csv(f"/home/tommaso/git_workspace/VAE_DefectedGraphene/analysis/original_df.csv", index=False)